In [1]:
import sys
import os
from tempfile import mkdtemp
sys.path.insert(0, '..')

import numpy as np
from process_bigraph import Composite, pp
from process_bigraph.experiments.parameter_scan import RunProcess

from biosimulator_processes import CORE 


Smoldyn is not properly installed in this environment and thus its process implementation cannot be registered. Please consult smoldyn documentation.
Available processes:
[ 'console-emitter',
  'ram-emitter',
  'composite',
  'cobra-process',
  'copasi-process',
  'tellurium-process',
  'utc-amici',
  'utc-copasi',
  'utc-tellurium',
  'copasi-step',
  'tellurium-step',
  'amici-step',
  'plotter',
  'plotter2d',
  'utc-comparator']


1. Define inputs

In [2]:
omex_dir = '../test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator'
omex_fp = omex_dir + '.omex'
sbml_fp = os.path.join(omex_dir, 'BIOMD0000000012_url.xml')
sedml_fp = os.path.join(omex_dir, 'simulation.sedml')
reports_path = os.path.join(omex_dir, 'reports.h5')
simulators = ['amici', 'copasi', 'tellurium']

2. Define composition document (manuscript?)

In [3]:
manuscript = {
    'amici': {
        '_type': 'step',
        'address': 'local:utc-amici',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['amici_floating_species_store'],
            'model_parameters': ['amici_model_parameters_store'],
            'reactions': ['amici_reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['amici_floating_species_store']
        }
    },
    'copasi': {
        '_type': 'step',
        'address': 'local:utc-copasi',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['copasi_floating_species_store'],
            'model_parameters': ['copasi_model_parameters_store'],
            'reactions': ['copasi_reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['copasi_floating_species_store']
        }
    },
    'tellurium': {
        '_type': 'step',
        'address': 'local:utc-tellurium',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['tellurium_floating_species_store'],
            'model_parameters': ['tellurium_model_parameters_store'],
            'reactions': ['tellurium_reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['tellurium_floating_species_store']
        }
    },
    'comparison': {
        '_type': 'step',
        'address': 'local:utc-comparator',
        'config': {
            'simulators': ['amici', 'copasi', 'tellurium'],
        },
        'inputs': {
            'time': ['time_store'],
            'amici_floating_species': ['amici_floating_species_store'],
            'copasi_floating_species': ['copasi_floating_species_store'],
            'tellurium_floating_species': ['tellurium_floating_species_store'],
        },
        'outputs': {
            'results': ['results_store'],
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'results': 'tree[string]',
            }
        },
        'inputs': {
            'results': ['results_store'],
        }
    }
}


output_bridge = {
    'inputs': {
        'results': ['results_store']
    },
    'outputs': {
        'results': ['results_store']
    }
}

3. Create composition with doc specifications

In [4]:
composition_config = {'state': manuscript, 'bridge': output_bridge}
composition = Composite(
    config=composition_config,
    core=CORE
)

2024-06-05 17:36:24.557 - amici.sbml_import - INFO - Finished importing SBML                         (4.47E-02s)
2024-06-05 17:36:24.611 - amici.sbml_import - INFO - Finished processing SBML observables            (5.11E-02s)
2024-06-05 17:36:24.616 - amici.sbml_import - INFO - Finished processing SBML event observables      (6.25E-07s)
2024-06-05 17:36:24.641 - amici.de_model - INFO - Finished computing xdot                            (3.73E-03s)
2024-06-05 17:36:24.648 - amici.de_model - INFO - Finished computing x0                              (3.53E-03s)
2024-06-05 17:36:24.669 - amici.de_model - INFO - Finished computing w                               (1.70E-02s)
2024-06-05 17:36:25.473 - amici.de_export - INFO - Finished generating cpp code                      (7.99E-01s)
2024-06-05 17:36:34.182 - amici.de_export - INFO - Finished compiling cpp code                       (8.71E+00s)


In [5]:
dir(composition)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'apply_updates',
 'bridge',
 'bridge_updates',
 'composition',
 'config',
 'config_schema',
 'core',
 'cycle_step_state',
 'determine_steps',
 'edge_paths',
 'emit_port',
 'emitter_paths',
 'expire_process_paths',
 'find_instance_paths',
 'front',
 'gather_results',
 'global_time_precision',
 'initial_state',
 'inputs',
 'interface',
 'invoke',
 'merge',
 'node_dependencies',
 'outputs',
 'process_paths',
 'process_schema',
 'process_update',
 'reset_step_state',
 'run',
 'run_process',
 'run_steps',
 'state',
 'step_dependencies',
 'step_paths',
 'step_triggers',
 'steps_remaining',
 'steps_run',
 'to_ru

In [6]:
pp(composition.state)

{ 'amici': { '_inputs': { 'floating_species': 'tree[string]',
                          'model_parameters': { 'KM': { '_apply': 'set',
                                                        '_type': 'float'},
                                                'a0_tr': { '_apply': 'set',
                                                           '_type': 'float'},
                                                'a_tr': { '_apply': 'set',
                                                          '_type': 'float'},
                                                'alpha': { '_apply': 'set',
                                                           '_type': 'float'},
                                                'alpha0': { '_apply': 'set',
                                                            '_type': 'float'},
                                                'average mRNA life time': { '_apply': 'set',
                                                                            '_typ

In [7]:
for simulator in simulators:
    print(f'{simulator}:')
    pp(composition.state[simulator]['instance'].initial_state())
    print('\n')


amici:
{ 'floating_species': { 'PX': 0.0,
                        'PY': 0.0,
                        'PZ': 0.0,
                        'X': 0.0,
                        'Y': 20.0,
                        'Z': 0.0},
  'model_parameters': { 'KM': 40.0,
                        'a0_tr': 0.0,
                        'a_tr': 0.0,
                        'alpha': 216.404,
                        'alpha0': 0.2164,
                        'average mRNA life time': 0.0,
                        'beta': 0.2,
                        'k_tl': 0.0,
                        'kd_mRNA': 0.0,
                        'kd_prot': 0.0,
                        'mRNA half life': 2.0,
                        'n': 2.0,
                        'protein half life': 10.0,
                        'tps_active': 0.5,
                        'tps_repr': 0.0005,
                        'translation efficiency': 20.0},
  'reactions': { 'degradation of CI': 0.0,
                 'degradation of CI transcripts': 0.0,
      

4. Run composition

In [8]:
composition.bridge

{'inputs': {'results': ['results_store']},
 'outputs': {'results': ['results_store']}}

In [9]:
composition.run(1)

In [10]:
composition_results = composition.gather_results()

In [11]:
composition_results

{('emitter',): []}

In [12]:
composition.state

{'time_store': [],
 'amici_floating_species_store': {'PX': None,
  'PY': None,
  'PZ': None,
  'X': None,
  'Y': None,
  'Z': None},
 'amici_model_parameters_store': {'beta': 0.0,
  'alpha0': 0.0,
  'alpha': 0.0,
  'translation efficiency': 0.0,
  'n': 0.0,
  'KM': 0.0,
  'mRNA half life': 0.0,
  'protein half life': 0.0,
  'average mRNA life time': 0.0,
  'kd_mRNA': 0.0,
  'kd_prot': 0.0,
  'k_tl': 0.0,
  'a_tr': 0.0,
  'tps_active': 0.0,
  'tps_repr': 0.0,
  'a0_tr': 0.0},
 'amici_reactions_store': {'degradation of LacI transcripts': 0.0,
  'degradation of TetR transcripts': 0.0,
  'degradation of CI transcripts': 0.0,
  'translation of LacI': 0.0,
  'translation of TetR': 0.0,
  'translation of CI': 0.0,
  'degradation of LacI': 0.0,
  'degradation of TetR': 0.0,
  'degradation of CI': 0.0,
  'transcription of LacI': 0.0,
  'transcription of TetR': 0.0,
  'transcription of CI': 0.0},
 'copasi_floating_species_store': {'PX': None,
  'PY': None,
  'PZ': None,
  'X': None,
  'Y': None,

In [13]:
from biosimulator_processes.processes.amici_process import UtcAmici
from biosimulator_processes.processes.copasi_process import UtcCopasi
from biosimulator_processes.processes.tellurium_process import UtcTellurium


utc_tellurium = UtcTellurium(config={'model': {'model_source': omex_dir + '.omex'}})
tellurium_results = utc_tellurium.update()

In [14]:
utc_copasi = UtcCopasi(config={'model': {'model_source': omex_dir + '.omex'}})
copasi_results = utc_copasi.update()

In [15]:
utc_amici = UtcAmici(config={'model': {'model_source': omex_dir + '.omex'}})
amici_results = utc_amici.update()

2024-06-05 17:36:34.597 - amici.sbml_import - INFO - Finished loading SBML                           (8.11E-03s)
2024-06-05 17:36:34.643 - amici.sbml_import - INFO - Finished importing SBML                         (4.35E-02s)
2024-06-05 17:36:34.661 - amici.sbml_import - INFO - Finished processing SBML observables            (1.35E-02s)
2024-06-05 17:36:34.665 - amici.sbml_import - INFO - Finished processing SBML event observables      (5.83E-07s)
2024-06-05 17:36:34.687 - amici.de_model - INFO - Finished computing xdot                            (3.82E-03s)
2024-06-05 17:36:34.694 - amici.de_model - INFO - Finished computing x0                              (3.55E-03s)
2024-06-05 17:36:34.705 - amici.de_model - INFO - Finished computing w                               (7.08E-03s)
2024-06-05 17:36:35.306 - amici.de_export - INFO - Finished generating cpp code                      (5.98E-01s)
2024-06-05 17:36:42.818 - amici.de_export - INFO - Finished compiling cpp code                  

In [16]:
comparison_input_state = {
    'amici_floating_species': amici_results['floating_species'],
    'copasi_floating_species': copasi_results['floating_species'],
    'tellurium_floating_species': tellurium_results['floating_species']
}

In [17]:
from biosimulator_processes.steps.comparator_step import UtcComparator


comparator = UtcComparator(config={'simulators': simulators})

In [18]:
comparator.update(comparison_input_state)

{'results': {'LacI protein': {'mse': {'copasi': {'copasi': 0.0,
     'amici': 3421.101751414077,
     'tellurium': 1112642.3515076651},
    'amici': {'copasi': 3421.101751414077,
     'amici': 0.0,
     'tellurium': 1122517.493697266},
    'tellurium': {'copasi': 1112642.3515076651,
     'amici': 1122517.493697266,
     'tellurium': 0.0}},
   'prox': {'copasi': {'copasi': True, 'amici': False, 'tellurium': False},
    'amici': {'copasi': False, 'amici': True, 'tellurium': False},
    'tellurium': {'copasi': False, 'amici': False, 'tellurium': True}}},
  'TetR protein': {'mse': {'copasi': {'copasi': 0.0,
     'amici': 3432.245484968864,
     'tellurium': 1182387.32612705},
    'amici': {'copasi': 3432.245484968864,
     'amici': 0.0,
     'tellurium': 1178485.6823350259},
    'tellurium': {'copasi': 1182387.32612705,
     'amici': 1178485.6823350259,
     'tellurium': 0.0}},
   'prox': {'copasi': {'copasi': True, 'amici': False, 'tellurium': False},
    'amici': {'copasi': False, 'amici